In [ ]:
import numpy as np
import pandas as pd
import vectorbtpro as vbt

In [ ]:
## Load m1 data
m1_data = vbt.HDFData.fetch('../../data/GU_OHLCV_3Y.h5')
m1_data.wrapper.index #pandas doaesn't recognise the frequency because of missing timestamps

In [ ]:
m5_data  = m1_data.resample('5T')   # Convert 1 minute to 5 mins
m15_data = m1_data.resample('15T')  # Convert 1 minute to 15 mins
m30_data = m1_data.resample('30T')  # Convert 1 minute to 30 mins
h1_data = m1_data.resample("1h")    # Convert 1 minute to 1 hour
h4_data = m1_data.resample("4h")    # Convert 1 minute to 4 hour
d1_data = m1_data.resample("1d")    # Convert 1 minute to daily

In [ ]:
# Obtain all the closing  prices using the .get() method
m5_close = m5_data.get()['Close']
m15_close = m15_data.get()['Close']

##m30 data
m30_open  = m30_data.get()['Open']
m30_close = m30_data.get()['Close']
m30_high  = m30_data.get()['High']
m30_low   = m30_data.get()['Low']

## h1 data
h1_open  = h1_data.get()['Open']
h1_close = h1_data.get()['Close']
h1_high  = h1_data.get()['High']
h1_low   = h1_data.get()['Low']

## h4 data
h4_open  = h4_data.get()['Open']
h4_close = h4_data.get()['Close']
h4_high  = h4_data.get()['High']
h4_low   = h4_data.get()['Low']

## 1D data
d1_open  = d1_data.get()['Open']
d1_close = d1_data.get()['Close']
d1_high  = d1_data.get()['High']
d1_low   = d1_data.get()['Low']

### Creating the Indicators for multi-timeframes

In [ ]:
## Method 1 - Manual of method of MTF Indicator creation
rsi_period = 21

## 5m indicators
m5_rsi = vbt.talib("RSI", timeperiod = rsi_period).run(m5_data.get("Close"), skipna=True).real
m5_bbands = vbt.talib("BBANDS").run(m5_data.get("Close"), skipna=True)
m5_bbands_rsi = vbt.talib("BBANDS").run(m5_rsi, skipna=True)

## 15m indicators
m15_rsi = vbt.talib("RSI", timeperiod = rsi_period).run(m15_data.get("Close"), skipna=True).real
m15_bbands = vbt.talib("BBANDS").run(m15_data.get("Close"), skipna=True)
m15_bbands_rsi = vbt.talib("BBANDS").run(m15_rsi, skipna=True)

## 30m indicators
m30_rsi = vbt.talib("RSI", timeperiod = rsi_period).run(m30_data.get("Close"), skipna=True).real
m30_bbands = vbt.talib("BBANDS").run(m30_data.get("Close"), skipna=True)
m30_bbands_rsi = vbt.talib("BBANDS").run(m30_rsi, skipna=True)

## h1 indicators
h1_rsi = vbt.talib("RSI", timeperiod = rsi_period).run(h1_data.get("Close"), skipna=True).real
h1_bbands = vbt.talib("BBANDS").run(h1_data.get("Close"), skipna=True)
h1_bbands_rsi = vbt.talib("BBANDS").run(h1_rsi, skipna=True)

## h4 indicators
h4_rsi = vbt.talib("RSI", timeperiod = rsi_period).run(h4_data.get("Close"), skipna=True).real
h4_bbands = vbt.talib("BBANDS").run(h4_data.get("Close"), skipna=True)
h4_bbands_rsi = vbt.talib("BBANDS").run(h4_rsi, skipna=True)

### Resample and creating a multi-time frame dataframe

Upsampling higher timeframe (`low frequency`) data to lower timeframe (`high frequency`) data

In [ ]:
def create_resamplers(result_dict_keys_list : list, source_indices : list,  source_frequencies :list, target_index : pd.Series, target_freq : str):
    """
    Creates a dictionary of vbtpro resampler objects.

    Parameters
    ==========
    result_dict_keys_list : list, list of strings, which are keys of the output dictionary
    source_indices        : list, list of pd.time series objects of the higher timeframes
    source_frequencies    : list, list of strings, which are short form representation of the time series order. Eg:["1D", "4h"]
    target_index          : pd.Series, target time series for the resampler objects
    target_freq           : str, target time frequency for the resampler objects,
                            use the 1st frequency in source_frequencies list

    Returns
    ===========
    resamplers_dict       : dict, vbt pro resampler objects
    """
    
    
    resamplers = []
    for si, sf in zip(source_indices, source_frequencies):
        resamplers.append(vbt.Resampler(source_index = si,  target_index = target_index,  source_freq = sf, target_freq = target_freq))
    return dict(zip(result_dict_keys_list, resamplers))

In [ ]:
## Create Resampler Objects for upsampling
src_indices = [m15_close.index, m30_close.index, h1_close.index, h4_close.index, d1_close.index ]
src_frequencies = ["15T","30T", "1h", "4h"] #, "1D"
resampler_dict_keys = ["m15_m5", "m30_m5", "h1_m5", "h4_m5"] # "d1_m5"

list_resamplers = create_resamplers(resampler_dict_keys, src_indices, src_frequencies, m5_close.index, "5T")
# list_resamplers = create_resamplers(resampler_dict_keys, src_indices, src_frequencies, m15_close.index, "15T")

list_resamplers

### Notes:
Setting the target index and targex frequency to 5min, creates more NULL values in the resampling of higher time frame data, which in turn creates a difference in the total nr. of orders ( `Total Trades` ) and ultimately the `Total Return [%]`

In [ ]:
## Initialize  dictionary
data = {}

# data = {"m5_close" : m5_close.ffill(), "m5_rsi" : m5_rsi.ffill(), 
#         "m5_bband_price_upper" : m5_bbands.upperband.ffill(),  "m5_bband_price_middle" : m5_bbands.middleband.ffill(),  "m5_bband_price_lower" : m5_bbands.lowerband.ffill(),
#         "m5_bband_rsi_upper" : m5_bbands_rsi.upperband.ffill(),  "m5_bband_rsi_middle" : m5_bbands_rsi.middleband.ffill(), "m5_bband_rsi_lower" : m5_bbands_rsi.lowerband.ffill() 
#         }

In [ ]:
## Use along with method 1 - Manual indicator creation for MTF
series_to_resample = [
    [m15_close, m15_rsi, m15_bbands.upperband, m15_bbands.middleband, m15_bbands.lowerband, m15_bbands_rsi.upperband, m15_bbands_rsi.middleband, m15_bbands_rsi.lowerband],
    [m30_close, m30_rsi, m30_bbands.upperband, m30_bbands.middleband, m30_bbands.lowerband, m30_bbands_rsi.upperband, m30_bbands_rsi.middleband, m30_bbands_rsi.lowerband],
    [h1_close, h1_rsi, h1_bbands.upperband, h1_bbands.middleband, h1_bbands.lowerband, h1_bbands_rsi.upperband, h1_bbands_rsi.middleband, h1_bbands_rsi.lowerband],
    [h4_close, h4_rsi, h4_bbands.upperband, h4_bbands.middleband, h4_bbands.lowerband, h4_bbands_rsi.upperband, h4_bbands_rsi.middleband, h4_bbands_rsi.lowerband],
    # [d1_close, d1_rsi, d1_bbands.upperband, d1_bbands.middleband, d1_bbands.lowerband, d1_bbands_rsi.upperband, d1_bbands_rsi.middleband, d1_bbands_rsi.lowerband]
    ]

data_keys = [
    ["m15_close", "m15_rsi", "m15_bband_price_upper",  "m15_bband_price_middle", "m15_bband_price_lower",  "m15_bband_rsi_upper",  "m15_bband_rsi_middle", "m15_bband_rsi_lower"], 
    ["m30_close", "m30_rsi", "m30_bband_price_upper",  "m30_bband_price_middle", "m30_bband_price_lower",  "m30_bband_rsi_upper",  "m30_bband_rsi_middle", "m30_bband_rsi_lower"],
    ["h1_close", "h1_rsi", "h1_bband_price_upper",  "h1_bband_price_middle",  "h1_bband_price_lower",  "h1_bband_rsi_upper",  "h1_bband_rsi_middle", "h1_bband_rsi_lower" ],
    ["h4_close", "h4_rsi", "h4_bband_price_upper",  "h4_bband_price_middle",  "h4_bband_price_lower",  "h4_bband_rsi_upper",  "h4_bband_rsi_middle", "h4_bband_rsi_lower" ],
    # ["d1_close", "d1_rsi", "d1_bband_price_upper",  "d1_bband_price_middle",  "d1_bband_price_lower",  "d1_bband_rsi_upper",  "d1_bband_rsi_middle", "d1_bband_rsi_lower" ]
         ]

In [ ]:
## Use this along with Method 1 of indicator creation

for lst_series, lst_keys, resampler in zip(series_to_resample, data_keys, resampler_dict_keys):
    for key, time_series in zip(lst_keys, lst_series):
        resampled_time_series = time_series.vbt.resample_closing(list_resamplers[resampler])
        data[key] = resampled_time_series.ffill()
        print('BEFORE RESAMPLING - ', key, time_series.index.freq, 'length:' ,len(time_series), 'NULL Count:',time_series.isna().sum())
        print("AFTER RESAMPLING - ",resampled_time_series.index.freq, 'length:' ,len(resampled_time_series), 'NULL Count:',resampled_time_series.isna().sum(), '\n' )

In [ ]:
## Add m30 OLH data
# data["m30_open"] = m30_open.vbt.resample_opening(list_resamplers['m30_m5']).ffill()
# data["m30_high"] = m30_high.vbt.resample_closing(list_resamplers['m30_m5']).ffill()
# data["m30_low"]  = m30_low.vbt.resample_closing(list_resamplers['m30_m5']).ffill()

## Add h1 OLH data
data["h1_open"] = h1_open.vbt.resample_opening(list_resamplers['h1_m5'])#.ffill()
data["h1_high"] = h1_high.vbt.resample_closing(list_resamplers['h1_m5'])#.ffill()
data["h1_low"]  = h1_low.vbt.resample_closing(list_resamplers['h1_m5'])#.ffill()

## Add h4 OLH data
data["h4_open"] = h4_open.vbt.resample_opening(list_resamplers['h4_m5'])#.ffill()
data["h4_high"] = h4_high.vbt.resample_closing(list_resamplers['h4_m5'])#.ffill()
data["h4_low"]  = h4_low.vbt.resample_closing(list_resamplers['h4_m5'])#.ffill()

Constructing DataFrame of Multi-Time Frame Data

In [ ]:
cols_order = [
    # 'm5_close','m5_rsi','m5_bband_price_upper','m5_bband_price_middle','m5_bband_price_lower','m5_bband_rsi_upper','m5_bband_rsi_middle','m5_bband_rsi_lower',
'm15_close','m15_rsi','m15_bband_price_upper','m15_bband_price_middle', 'm15_bband_price_lower','m15_bband_rsi_upper','m15_bband_rsi_middle', 'm15_bband_rsi_lower',
'm30_close','m30_rsi', 'm30_bband_price_upper', 'm30_bband_price_middle', 'm30_bband_price_lower', 'm30_bband_rsi_upper', 'm30_bband_rsi_middle', 'm30_bband_rsi_lower',
 'h1_open', 'h1_high', 'h1_low', 'h1_close', 'h1_rsi', 'h1_bband_price_upper', 'h1_bband_price_middle', 'h1_bband_price_lower', 'h1_bband_rsi_upper', 'h1_bband_rsi_middle','h1_bband_rsi_lower',
 'h4_open', 'h4_high', 'h4_low', 'h4_close', 'h4_rsi', 'h4_bband_price_upper', 'h4_bband_price_middle', 'h4_bband_price_lower', 'h4_bband_rsi_upper', 'h4_bband_rsi_middle', 'h4_bband_rsi_lower'
 ]

In [ ]:
## construct a multi-timeframe dataframe
mtf_df = pd.DataFrame(data)[cols_order]
print("Length of mtf_df:",len(mtf_df), f'on {mtf_df.index.freq} frequency')

In [ ]:
for col in mtf_df.columns:
    time_series = mtf_df[col]
    print(col,time_series.index.freq, 'length:' ,len(time_series), 'NULL Count:',time_series.isna().sum())

In [ ]:
display(mtf_df)

In [ ]:
# print(mtf_df.columns.to_list())

In [ ]:
mtf_df.info()

### Double Bollinger Band - Strategy Conditions
The trading conditions (rules) of the strategy are as follows:

1. A long (buy) signal is generated whenever the H4 market (Low) price surpasses its lower Bollinger band after having been below it while simultaneously, the 5m RSI surpasses its lower Bollinger band after having been below it.

2. A short (sell) signal is generated whenever the H4 market (High) price breaks its upper Bollinger band after having been above it while simultaneously, the 5m RSI breaks below its upper Bollinger band after having been above it.

In [ ]:
required_cols = [
                # 'm5_close', "m5_rsi" , "m5_bband_rsi_lower", "m5_bband_rsi_upper" ,
                'm15_close','m15_rsi','m15_bband_rsi_lower', 'm15_bband_rsi_upper',
                #  "h1_low", "h1_rsi" , "h1_bband_price_lower" ,
                 'h4_low', "h4_rsi", "h4_bband_price_lower", "h4_bband_price_upper"]

In [ ]:
mtf_df[required_cols][(mtf_df['h4_low'] < mtf_df['h4_bband_price_lower'])]

In [ ]:
## Higher values greater than 1.0 are like moving up the lower RSI b-band, signifying if the lowerband rsi is anywhere around 1% of the lower b-band validate that case as True
bb_upper_fract = 0.99
bb_lower_fract = 1.01

In [ ]:
# mtf_df[required_cols][(mtf_df['m5_rsi'] < (bb_lower_fract * mtf_df['m5_bband_rsi_lower']) )]

In [ ]:
## When additional checking if RSI goes below 30 it Yields very few results
# mtf_df[required_cols][(mtf_df['m5_rsi'] < (bb_lower_fract * mtf_df['m5_bband_rsi_lower']) ) & (mtf_df['m5_rsi'] <= 30)]

In [ ]:
## Long Entry Conditions
c1_long_entry = (mtf_df['h4_low'] <= mtf_df['h4_bband_price_lower'])
c2_long_entry = (mtf_df['m15_rsi'] <= (bb_lower_fract * mtf_df['m15_bband_rsi_lower']) )
# c3_long_entry = (mtf_df['m5_rsi'] <= 30)
# c4_long_entry = (mtf_df['m5_rsi'] <= (bb_lower_fract * mtf_df['m5_bband_rsi_lower']) )

## Long Exit Conditions
c1_long_exit =  (mtf_df['h4_high'] >= mtf_df['h4_bband_price_upper'])
c2_long_exit = (mtf_df['m15_rsi'] >= (bb_upper_fract * mtf_df['m15_bband_rsi_upper'])) 
# c3_long_exit = (mtf_df['m5_rsi'] >= 70)
# c4_long_exit = (mtf_df['m5_rsi'] >= (bb_upper_fract * mtf_df['m5_bband_rsi_upper']))

In [ ]:
## Strategy conditions check - Using m15 and h4 data 
mtf_df['entry'] = c1_long_entry & c2_long_entry
mtf_df['exit']  = c1_long_exit & c2_long_exit

In [ ]:
## Strategy conditions check - Using m5, m15 and h4 data 
# mtf_df['entry'] = c1_long_entry & c2_long_entry & c3_long_entry
# mtf_df['exit']  = c1_long_exit & c2_long_exit & c3_long_exit

In [ ]:
## Strategy conditions check - Using m5 (2 conditions) and h4 data 
# mtf_df['entry'] = c1_long_entry & c3_long_entry & c4_long_entry
# mtf_df['exit']  = c1_long_exit & c3_long_exit & c4_long_exit

In [ ]:
## Final conditions - Using m5 (1 condition) and h4 data 
# mtf_df['entry'] = c1_long_entry & c4_long_entry
# mtf_df['exit']  = c1_long_exit & c4_long_exit

In [ ]:
mtf_df['signal'] = 0   
mtf_df['signal'] = np.where( mtf_df['entry'] ,1, 0)
mtf_df['signal'] = np.where( mtf_df['exit'] ,-1, mtf_df['signal'])

In [ ]:
entries = mtf_df.signal == 1.0
exits = mtf_df.signal == -1.0

# mtf_df['short_entries'] = short_entries = long_exits
# mtf_df['short_exits'] = short_exits = long_entries

# mtf_df.drop(columns="signal", axis=1, inplace = True)
# mtf_df.head()

_Cleaning_ `entries` and `exit` signals by removing redundant signals

In [ ]:
print(entries.vbt.signals.total(), exits.vbt.signals.total())
print(len(entries), len(exits))

In [ ]:
## Clean redundant and duplicate signals
clean_entries, clean_exits = entries.vbt.signals.clean(exits)

In [ ]:
print(entries.vbt.signals.total(),exits.vbt.signals.total())
print(len(clean_entries), len(clean_exits))

Resampling `entries` and `exits` to `H4` for plotting purpose

In [ ]:
%%time
## Faster method
h4_entries = entries.vbt.resample_apply("4h", "any", wrap_kwargs=dict(dtype=bool))
h4_exits = exits.vbt.resample_apply("4h", "any", wrap_kwargs=dict(dtype=bool))

In [ ]:
print(len(h4_entries),len(h4_exits))
print(h4_entries.vbt.signals.total(),h4_exits.vbt.signals.total())

In [ ]:
clean_h4_entries, clean_h4_exits = h4_entries.vbt.signals.clean(h4_exits)

In [ ]:
print(len(clean_h4_entries),len(clean_h4_exits))
print(clean_h4_entries.vbt.signals.total(),clean_h4_exits.vbt.signals.total())

### Plotting Indicators

In [ ]:
## Global Plot Settings
vbt.settings.set_theme("dark")
vbt.settings['plotting']['layout']['width'] = 1280

In [ ]:
## Acquire Data for Indicators Plot
h4_df = h4_data.get()
h4_rsi = vbt.talib("RSI", timeperiod = rsi_period).run(h4_data.get("Close"), skipna=True).real
h4_bbands = vbt.talib("BBANDS").run(h4_data.get("Close"), skipna=True)
h4_bbands_rsi = vbt.talib("BBANDS").run(h4_rsi, skipna=True)
h4_df

Create Backtest Simulation for 2BB Strategy using `vbt.Portfolio.from_signals()`

In [ ]:
pf_2_bb_custom = vbt.Portfolio.from_signals(
    mtf_df['m15_close'], 
    # mtf_df['m5_close'], 
    entries=entries, 
    exits=exits, 
    direction = "both",
    freq=pd.Timedelta(minutes=5), 
    init_cash=100000
)

In [ ]:
pf_2_bb_custom.stats()

### Questions:
1.) Where do I see the Max Drawdown Duration as being 282 days?

In [ ]:
# pf_2_bb_custom.plot().show() ## This takes too long (30 secs) as it uses 5m timeframe
pf_2_bb_custom.resample("1d").plot().show()

In [ ]:
# pf_2_bb_custom.orders.plot(xaxis=dict(rangeslider_visible=True),**kwargs1).show()
kwargs1 = {"title_text" : "Orders - Stats & Plot", "title_font_size" : 18}
pf_2_bb_custom.orders.resample("1d").plot(xaxis=dict(rangeslider_visible=True),**kwargs1).show()

In [ ]:
## Simple line plots seem to be using matplotlib
pf_2_bb_custom.trades.records_readable.PnL.cumsum().plot()

In [ ]:
pnl_series = pf_2_bb_custom.trades.pnl.to_pd()
pnl_series[pnl_series.notna()]

### Creating Stacked SubPlots on Sliced Data

In [ ]:
def plot_strategy(slice_lower : str, slice_upper: str, df : pd.DataFrame , rsi : pd.Series,
                         bb_price : vbt.indicators.factory, bb_rsi : vbt.indicators.factory, 
                         entries: pd.Series, exits: pd.Series, 
                         long_trades: pd.DataFrame, short_trades: pd.DataFrame,
                         pf: vbt.portfolio.base.Portfolio,
                         show_legend : bool = True):
    """Creates a stacked indicator plot for the 2BB strategy.
    Parameters
    ===========
    slice_lower : str, start date of dataframe slice in yyyy.mm.dd format
    slice_upper : str, start date of dataframe slice in yyyy.mm.dd format
    df          : pd.DataFrame, containing the OHLCV data
    rsi         : pd.Series, rsi indicator time series in same freq as df
    bb_price    : vbt.indicators.factory.talib('BBANDS'), computed on df['close'] price
    bb_rsi      : vbt.indicators.factory.talib('BBANDS') computer on RSI
    entries     : pd.Series, time series data of long entries
    exits       : pd.Series, time series data of long exits
    long_trades : pd.DataFrame, long_trades from `pf.trades.records_readable`
    short_trades: pd.DataFrame, short_trades from `pf.trades.records_readable`
    show_legend : bool, switch to show or completely hide the legend box on the plot
    
    Returns
    =======
    fig         : plotly figure object
    """
    kwargs1 = {"title_text" : "H4 OHLCV with BBands on Price and RSI", 
               "title_font_size" : 18,
               "height" : 960,
               "legend" : dict(yanchor="top",y=0.99, xanchor="right",x= 0.1)}
    fig = vbt.make_subplots(rows=2,cols=1, shared_xaxes=True, vertical_spacing=0.1)
    ## Filter Data according to date slice
    df_slice = df[["Open", "High", "Low", "Close"]][slice_lower : slice_upper]
    bb_price = bb_price[slice_lower : slice_upper]
    rsi = rsi[slice_lower : slice_upper]
    bb_rsi = bb_rsi[slice_lower : slice_upper]
    entries = entries[slice_lower : slice_upper]
    exits = exits[slice_lower : slice_upper]
    long_trades = long_trades[slice_lower : slice_upper].reset_index()
    short_trades = short_trades[slice_lower : slice_upper].reset_index()

    ## Retrieve Filled entry and exit prices
    entry_prices = pd.concat([long_trades[['Entry Index', 'Avg Entry Price']], short_trades[['Entry Index', 'Avg Entry Price']]]).sort_values('Entry Index').set_index('Entry Index')
    exit_prices = pd.concat([long_trades[['Exit Index', 'Avg Exit Price']], short_trades[['Exit Index', 'Avg Exit Price']]]).sort_values('Exit Index').set_index('Exit Index')
    # entry_prices.index.names = ['time']
    # exit_prices.index.names = ['time']
    # entry_prices.name = 'Entries'
    # exit_prices.name = 'Exits'
    
    ## Retrieve datetime index of rows where price data is NULL
    # retrieve the dates that are in the original datset
    dt_obs = df_slice.index.to_list()
    # Drop rows with missing values
    dt_obs_dropped = df_slice['Close'].dropna().index.to_list()
    # store  dates with missing values
    dt_breaks = [d for d in dt_obs if d not in dt_obs_dropped]

    ## Plot Figures
    df_slice.vbt.ohlcv.plot(add_trace_kwargs=dict(row=1, col=1),  fig=fig, **kwargs1) ## Without Range Slider
    rsi.rename("RSI").vbt.plot(add_trace_kwargs=dict(row=2, col=1), trace_kwargs = dict(connectgaps=True), fig=fig, **kwargs1 ) 

    bb_dotted_line = dict(color="white",width=1, dash="dot")
    bb_price.plot(add_trace_kwargs=dict(row=1, col=1),fig=fig, **kwargs1,
                lowerband_trace_kwargs=dict(fill=None, name = 'BB_Price_Lower', connectgaps=True, line = bb_dotted_line), 
                upperband_trace_kwargs=dict(fill=None, name = 'BB_Price_Upper', connectgaps=True, line = bb_dotted_line),
                middleband_trace_kwargs=dict(fill=None, name = 'BB_Price_Middle', connectgaps=True) )

    bb_rsi.plot(add_trace_kwargs=dict(row=2, col=1),limits=(25, 75),fig=fig,
                lowerband_trace_kwargs=dict(fill=None, name = 'BB_RSI_Lower', connectgaps=True,line = bb_dotted_line), 
                upperband_trace_kwargs=dict(fill=None, name = 'BB_RSI_Upper', connectgaps=True,line = bb_dotted_line),
                middleband_trace_kwargs=dict(fill=None, name = 'BB_RSI_Middle', connectgaps=True, visible = False))
    
    pf[slice_lower:slice_upper].plot_trade_signals(add_trace_kwargs=dict(row=1, col=1),fig=fig,
                                                   plot_close=False, plot_positions="lines")

    ## Add Marker style for Long Entries and Long Exits
    # long_entry_marker_style = dict(color="limegreen",symbol="triangle-up",size = 14)
    # long_exit_marker_style_zero = dict(color="red", symbol="square-open", size = 14)
    # long_exit_marker_style_profit = dict(color="red", symbol="triangle-down", size = 14)
    # long_exit_marker_style_loss = dict(color="red", symbol="circle-open-dot", size = 14)

    pf.trades.direction_long[slice_lower : slice_upper].plot(
                                        add_trace_kwargs=dict(row=1, col=1),fig=fig,
                                        plot_close = False,
                                        plot_markers = False,
                                        # entry_trace_kwargs = dict(name = "Long Entries", marker = long_entry_marker_style),
                                        # exit_trace_kwargs = dict(name = "Long Exits BreakEven", marker = long_exit_marker_style_zero),
                                        # exit_profit_trace_kwargs = dict(name = "Long Exits Profit", marker = long_exit_marker_style_profit),
                                        # exit_loss_trace_kwargs = dict(name = "Long Exits Loss", marker = long_exit_marker_style_loss),
                                        )
                                        
    ## Add Marker Style for Short Entries and Short Exits
    # shot_entry_marker_style = dict(color="limegreen",symbol="triangle-down",size = 14)
    # short_exit_marker_style_zero = dict(color="red", symbol="square-open", size = 14)
    # short_exit_marker_style_profit = dict(color="red", symbol="triangle-up", size = 14)
    # short_exit_marker_style_loss = dict(color="red", symbol="circle-open-dot", size = 14)

    pf.trades.direction_short[slice_lower : slice_upper].plot(
                                            add_trace_kwargs=dict(row=1, col=1),fig=fig,
                                            plot_close = False,
                                            plot_markers = False,
                                            # entry_trace_kwargs = dict(name = "Short Entries", marker = shot_entry_marker_style),
                                            # exit_trace_kwargs = dict(name = "Short Exits BreakEven", marker = short_exit_marker_style_zero),
                                            # exit_profit_trace_kwargs = dict(name = "Short Exits Profit", marker = short_exit_marker_style_profit),
                                            # exit_loss_trace_kwargs = dict(name = "Short Exits Loss", marker = short_exit_marker_style_loss),
                                            )

    ## Add Entries and Long Exits on Price
    # entries.vbt.signals.plot_as_entries(df_slice['Close'], fig=fig,
    #                                     trace_kwargs=dict(marker=dict(color="limegreen", size = 12), 
    #                                                       name="Entries")
    #                                         )
    # exits.vbt.signals.plot_as_exits(df_slice['Close'], fig = fig,
    #                                     trace_kwargs=dict(marker=dict(color="red", size = 12), 
    #                                                     name="Exits") 
    #                                     )

    ## Add Entries and Long Exits on RSI in lower subplot
    # entries.vbt.signals.plot_as_entries(rsi, fig = fig,
    #                                          add_trace_kwargs=dict(row=2, col=1),
    #                                          trace_kwargs=dict(marker=dict(color="limegreen"),showlegend = False))  

    # exits.vbt.signals.plot_as_exits(rsi, fig = fig,showlegend = False,
    #                                      add_trace_kwargs=dict(row=2, col=1),
    #                                      trace_kwargs=dict(marker=dict(color="red"),showlegend = False))

    ## Dotted Lines for Long Trades
    # x_axis_long_entries = long_trades['Entry Index'].map(lambda x : x.replace(microsecond=0))
    # y_axis_long_entries = long_trades['Avg Entry Price'].values
    # x_axis_long_exits = long_trades['Exit Index'].map(lambda x : x.replace(microsecond=0))
    # y_axis_long_exits = long_trades['Avg Exit Price'].values

    # for entry_time, entry_price, exit_time, exit_price in zip(x_axis_long_entries, y_axis_long_entries, 
    #                                                             x_axis_long_exits, y_axis_long_exits):
    #     fig.add_shape(type="line",x0=entry_time, y0=entry_price, 
    #                                 x1=exit_time, y1=exit_price,
    #                                 line=dict(color="aquamarine",width=2,dash="dot"))

    ## Dotted Lines for Short Trades
    # x_axis_short_entries = short_trades['Entry Index'].map(lambda x : x.replace(microsecond=0))
    # y_axis_short_entries = short_trades['Avg Entry Price'].values
    # x_axis_short_exits = short_trades['Exit Index'].map(lambda x : x.replace(microsecond=0))
    # y_axis_short_exits = short_trades['Avg Exit Price'].values

    # for entry_time, entry_price, exit_time, exit_price in zip(x_axis_short_entries, y_axis_short_entries, 
    #                                                             x_axis_short_exits, y_axis_short_exits):
    #     fig.add_shape(type="line",x0=entry_time, y0=entry_price, 
    #                                 x1=exit_time, y1=exit_price,
    #                                 line=dict(color="magenta",width=2, dash="dot")) 

    fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])
    fig.layout.showlegend = show_legend  
    # fig.write_html(f"2BB_Strategy_{slice_lower}_to_{slice_upper}.html")
    
    return fig

In [ ]:
pf_2_bb_custom.trade_history

In [ ]:
df_trades = pf_2_bb_custom.trades.records_readable
df_trades

In [ ]:
long_trades = df_trades[['Position Id','Entry Index', 'Avg Entry Price', 'Exit Index', 'Avg Exit Price' , 'Direction' ,'PnL', 'Return']][df_trades['Direction'] == "Long"].set_index('Entry Index')#.squeeze()
short_trades = df_trades[['Position Id','Entry Index', 'Avg Entry Price', 'Exit Index', 'Avg Exit Price', 'Direction' ,'PnL', 'Return']][df_trades['Direction'] == "Short"].set_index('Entry Index')#.squeeze()

In [ ]:
len(long_trades), len(short_trades)

In [ ]:
slice_lower = '2019.09.01'
slice_higher = '2020.01.31'

In [ ]:
## from VBT Pro - develop branch
# pf_2_bb_custom.plot_trade_signals()

In [ ]:
## from VBT Pro - develop branch
pf_2_bb_custom[slice_lower:slice_higher].plot_trade_signals(plot_close=False, plot_positions="lines").show()

In [ ]:
## If you don't add .show() - white border will be present
pf_2_bb_custom.trades[slice_lower : slice_higher].plot(plot_close = False)

API Documentation:
```python
Trades.plot(
    column=None,
    plot_ohlc=True,
    plot_close=True,
    plot_markers=True,
    plot_zones=True,
    ohlc_type=None,
    ohlc_trace_kwargs=None,
    close_trace_kwargs=None,
    entry_trace_kwargs=None,
    exit_trace_kwargs=None,
    exit_profit_trace_kwargs=None,
    exit_loss_trace_kwargs=None,
    active_trace_kwargs=None,
    profit_shape_kwargs=None,
    loss_shape_kwargs=None,
    add_trace_kwargs=None,
    xref='x',
    yref='y',
    fig=None,
    **layout_kwargs
)
```

In [ ]:
## Add Marker Style for Long Entries and Long Exits
long_entry_marker_style = dict(color="limegreen",symbol="triangle-up",size = 12)
long_exit_marker_style_zero = dict(color="red", symbol="square-open", size = 12)
long_exit_marker_style_profit = dict(color="red", symbol="triangle-down", size = 12)
long_exit_marker_style_loss = dict(color="red", symbol="circle-open-dot", size = 12)

In [ ]:
## Long Entries and Long Exits - entry_trace_kwargs and exit_trace_kwargs NOT WORKING
pf_2_bb_custom.trades.direction_long[slice_lower : slice_higher].plot(
                                        plot_close = False,
                                        entry_trace_kwargs = dict(name = "Long Entries", marker = long_entry_marker_style),
                                        exit_trace_kwargs = dict(name = "Long Exits BreakEven", marker = long_exit_marker_style_zero),
                                        exit_profit_trace_kwargs = dict(name = "Long Exits Profit", marker = long_exit_marker_style_profit),
                                        exit_loss_trace_kwargs = dict(name = "Long Exits Loss", marker = long_exit_marker_style_loss),
                                                        ).show()

In [ ]:
## Add Marker Style for Short Entries and Short Exits
shot_entry_marker_style = dict(color="limegreen",symbol="triangle-down",size = 12)
short_exit_marker_style_zero = dict(color="red", symbol="square-open", size = 12)
short_exit_marker_style_profit = dict(color="red", symbol="triangle-up", size = 12)
short_exit_marker_style_loss = dict(color="red", symbol="circle-open-dot", size = 12)

In [ ]:
pf_2_bb_custom.trades.direction_short[slice_lower : slice_higher].plot(
    plot_close = False,
    # plot_zones = False,
    # exit_profit_trace_kwargs = dict(visible = False),
    # exit_loss_trace_kwargs = dict(visible = False),
    entry_trace_kwargs = dict(name = "Short Entries", marker = shot_entry_marker_style),
    exit_trace_kwargs = dict(name = "Short Exits BreakEven", marker = short_exit_marker_style_zero),
    exit_profit_trace_kwargs = dict(name = "Short Exits Profit", marker = short_exit_marker_style_profit),
    exit_loss_trace_kwargs = dict(name = "Short Exits Loss", marker = short_exit_marker_style_loss),
).show()

In [ ]:
slice_lower = '2019.09.01'
slice_higher = '2019.12.31'

fig = plot_strategy(slice_lower, slice_higher, h4_df, h4_rsi, 
                           h4_bbands, h4_bbands_rsi, 
                           clean_h4_entries, clean_h4_exits, 
                           long_trades, short_trades,
                           pf_2_bb_custom,
                           show_legend = True)

fig.show()

### Save final dataframe with consolidated signals and `P&L`

In [ ]:
# mtf_df['cleaned_signal'] = signal
mtf_df["pnl"] = pnl_series
mtf_df

In [ ]:
# mtf_df.to_csv("2BB_Final_Simulation.csv", index = False)

In [ ]:
mtf_df.columns